In [8]:
# Basic imports

import torch
from transformers import pipeline, set_seed
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [9]:
generator = pipeline("text-generation", model="gpt2")
set_seed(42)

prompt = "The future of Artificial Intelligence in education is"
results = generator(prompt, max_length=80, num_return_sequences=2)

for i, r in enumerate(results):
    print(f"\nGenerated Text {i+1}:\n", r['generated_text'])


Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Generated Text 1:
 The future of Artificial Intelligence in education is still in its infancy, but with a lot of work to do, we can expect the future of AI to be much more complex.

You can read the full article here.

Generated Text 2:
 The future of Artificial Intelligence in education is far from clear. The question is whether an AI may or may not be an intelligent person. This is not so much a question of whether an AI could be a human for reasons of convenience, as whether an AI has the ability to understand and understand human language (or speech). This has not yet been demonstrated. However, there is little doubt that a human is capable of understanding and understanding human language.

This is not all, though. Most likely, AI will be used in everyday tasks, and will probably be useful to a variety of different people in different capacities. It is not clear yet why, for example, humans will prefer to use computers to do tasks as complex as writing a book, or why they may not

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# ----------------------------
# Training text
# ----------------------------
text = (
    "Generative models are powerful tools for creating text. "
    "They can learn patterns from a dataset and generate new content. "
    "LSTM networks are one type of model used in text generation. "
    "Text generation has applications in chatbots, story writing, and content creation. "
    "Machine learning allows computers to understand sequences and predict next words. "
    "Training on more data improves the quality of generated text. "
    "Using word-level tokenization produces more readable sentences. "
    "AI is transforming education, healthcare, and creative industries. "
    "Models like GPT-2 are trained on massive datasets to generate coherent paragraphs. "
    "This demo shows how a small LSTM can generate text word by word."
)

# ----------------------------
# Tokenize words
# ----------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# ----------------------------
# Create sequences
# ----------------------------
input_sequences = []
for line in text.split('. '):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

# Split predictors and label
X = input_sequences[:, :-1]
y = tf.keras.utils.to_categorical(input_sequences[:, -1], num_classes=total_words)

# ----------------------------
# Build model
# ----------------------------
model = Sequential([
    layers.Embedding(total_words, 32, input_length=max_seq_len-1),
    layers.LSTM(64),
    layers.Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam')

# ----------------------------
# Train model
# ----------------------------
model.fit(X, y, epochs=100, verbose=0)

# ----------------------------
# Text generation with temperature
# ----------------------------
def generate_text(seed_text, next_words=30, temperature=0.7):
    result = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([result])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        preds = model.predict(token_list, verbose=0)[0]
        
        # Apply temperature sampling
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        
        next_index = np.random.choice(len(preds), p=preds)
        output_word = tokenizer.index_word.get(next_index, '')
        result += ' ' + output_word
    return result

# ----------------------------
# Generate sample text
# ----------------------------
print(generate_text("Generative models", next_words=30, temperature=0.7))


Generative models models trained model for for text text generation paragraphs paragraphs text by datasets powerful to generative one next creation new words generation new applications creation transforming tools understand words are
